In [1]:
import numpy as np
from asdf import AsdfFile
from astropy.io import fits
from astropy import wcs as astwcs
from astropy import table
from gwcs import wcs

from jwst import datamodels
from jwst.assign_wcs import nirspec
from jwst.transforms import models

In [2]:
#Create reference file dict
def create_reference_files(datamodel):
    """
    Create a dict {reftype: reference_file}.
    """
    refs = {}
    step = assign_wcs_step.AssignWcsStep()
    for reftype in assign_wcs_step.AssignWcsStep.reference_file_types:
        refs[reftype] = step.get_reference_file(datamodel, reftype)
    return refs

# These are the CV3 files
refs = {'camera': '/grp/crds/cache/references/jwst/jwst_nirspec_camera_0004.asdf',
        'collimator': '/grp/crds/cache/references/jwst/jwst_nirspec_collimator_0004.asdf',
        'disperser': '/grp/crds/cache/references/jwst/jwst_nirspec_disperser_0035.asdf',
        'distortion': 'N/A',
        'filteroffset': 'N/A',
        'fore': '/grp/crds/cache/references/jwst/jwst_nirspec_fore_0022.asdf',
        'fpa': '/grp/crds/cache/references/jwst/jwst_nirspec_fpa_0005.asdf',
        'ifufore': '/grp/crds/cache/references/jwst/jwst_nirspec_ifufore_0003.asdf',
        'ifupost': 'ifupost.asdf',
        'ifuslicer': '/grp/crds/cache/references/jwst/jwst_nirspec_ifuslicer_0003.asdf',
        'msa': '/grp/crds/cache/references/jwst/jwst_nirspec_msa_0005.asdf',
        'ote': 'ote.asdf',
        'regions': 'N/A',
        'specwcs': 'N/A',
        'wavelengthrange': '/grp/crds/cache/references/jwst/jwst_nirspec_wavelengthrange_0004.asdf'}


In [3]:
wcs_kw = {'wcsaxes': 2, 'ra_ref': 165, 'dec_ref': 54,
          'v2_ref': -8.3942412, 'v3_ref': -5.3123744, 'roll_ref': 37,
          'crpix1': 1024, 'crpix2': 1024,
          'cdelt1': .08, 'cdelt2': .08,
          'ctype1': 'RA---TAN', 'ctype2': 'DEC--TAN',
          'pc1_1': 1, 'pc1_2': 0, 'pc2_1': 0, 'pc2_2': 1
          }

slit_fields_num = ["shutter_id", "xcen", "ycen",
                   "ymin", "ymax", "quadrant", "source_id",
                   "stellarity", "source_xpos", "source_ypos"]


slit_fields_str = ["name", "shutter_state", "source_name", "source_alias"]


In [19]:
t = table.Table.read('/home/dencheva/data/astropy/test-nrs/nrs_testing/test_model/msa_functional_ESA_v1_20180619.txt', format='ascii')
#t.columns

In [5]:
def create_hdul(detector='NRS1'):
    """
    Create a fits HDUList instance.
    """
    hdul = fits.HDUList()
    phdu = fits.PrimaryHDU()
    phdu.header['instrume'] = 'NIRSPEC'
    phdu.header['detector'] = detector
    phdu.header['time-obs'] = '8:59:37'
    phdu.header['date-obs'] = '2016-09-05'

    scihdu = fits.ImageHDU()
    scihdu.header['EXTNAME'] = "SCI"
    for item in wcs_kw.items():
        scihdu.header[item[0]] = item[1]
    hdul.append(phdu)
    hdul.append(scihdu)
    return hdul

In [6]:
def create_nirspec_mos_file(grating, filter, lamp="N/A", detector="NRS1"):
    image = create_hdul(detector)
    image[0].header['exp_type'] = 'NRS_MSASPEC'
    image[0].header['filter'] = filter
    image[0].header['grating'] = grating
    image[0].header['crval3'] = 0
    image[0].header['wcsaxes'] = 3
    image[0].header['ctype3'] = 'WAVE'
    image[0].header['pc3_1'] = 1
    image[0].header['pc3_2'] = 0
    return image

In [7]:
hdul = create_nirspec_mos_file(grating='G395H', filter='F290LP',  detector='NRS1')
im = datamodels.ImageModel(hdul)

slit = models.Slit(name=1, shutter_id=4699, xcen=319, ycen=13, ymin=-0.55000000000000004,
                   ymax=0.55000000000000004, quadrant=3, source_id=1, shutter_state='x',
                   source_name='lamp', source_alias='foo', stellarity=100.0, source_xpos=-0.5,
                   source_ypos=0.5)
open_slits = [slit]


pipeline = nirspec.slitlets_wcs(im, refs, open_slits)
w = wcs.WCS(pipeline)
im.meta.wcs = w


2018-06-20 11:03:57,840 - stpipe - INFO - gwa_ytilt is None deg
2018-06-20 11:03:57,841 - stpipe - INFO - gwa_xtilt is None deg
2018-06-20 11:03:57,842 - stpipe - INFO - gwa_xtilt not applied
2018-06-20 11:03:57,842 - stpipe - INFO - gwa_ytilt not applied
2018-06-20 11:03:57,864 - stpipe - INFO - SPORDER= -1, wrange=[2.87e-06, 5.27e-06]
2018-06-20 11:03:58,295 - stpipe - INFO - There are 0 open slits in quadrant 1
2018-06-20 11:03:58,296 - stpipe - INFO - There are 0 open slits in quadrant 2
2018-06-20 11:03:58,297 - stpipe - INFO - There are 1 open slits in quadrant 3
2018-06-20 11:03:58,297 - stpipe - INFO - Getting slits location for quadrant 3
2018-06-20 11:03:58,458 - stpipe - INFO - There are 0 open slits in quadrant 4
2018-06-20 11:03:58,459 - stpipe - INFO - There are 0 open slits in quadrant 5


In [8]:
# Setup the test
slitx = [0] * 5
slity = [-.5, -.25, 0, .25, .5]
wave_range = [2.87e-06, 5.27e-06]
lam = np.array([2.9, 3.39, 3.88, 4.37, 5]) * 10**-6

# Use slit S200A1
slit_wcs = nirspec.nrs_wcs_set_input(im, 1)

In [9]:
# Slit to MSA absolute
slit2msa = slit_wcs.get_transform('slit_frame', 'msa_frame')
msax, msay, _= slit2msa(slitx, slity, lam)
print('slitx: ', slitx )
print('slity: ', slity)
print('msax: ', msax)
print('msay: ', msay)

print('diff slitx', slitx - t['xslitpos'])
print('diff slity', slity - t['yslitpos'])
print('diff x_msa', msax - t['xmsapos'])
print('diff y_msa', msay - t['ymaspos'])

slitx:  [0, 0, 0, 0, 0]
slity:  [-0.5, -0.25, 0, 0.25, 0.5]
msax:  [ 0.03797902  0.03797902  0.03797903  0.03797904  0.03797905]
msay:  [-0.03950002 -0.03945627 -0.03941252 -0.03936877 -0.03932502]
diff slitx xslitpos
--------
       0
       0
       0
       0
       0
diff slity yslitpos
--------
     0.0
     0.0
     0.0
     0.0
     0.0
diff x_msa xmsapos
-------
    0.0
    0.0
    0.0
    0.0
    0.0
diff y_msa ymaspos
-------
    0.0
    0.0
    0.0
    0.0
    0.0


In [10]:
# MSA to GWA entrance
# This runs the Collimator forward, Unitless to Directional cosine, and 3D Rotation
disp = datamodels.DisperserModel(refs['disperser'])
disperser = nirspec.correct_tilt(disp, im.meta.instrument.gwa_xtilt, im.meta.instrument.gwa_ytilt)
collimator2gwa = nirspec.collimator_to_gwa(refs, disperser)
x_gwa_in, y_gwa_in, z_gwa_in = collimator2gwa(msax, msay)
print('x_gwa_entrance:' , x_gwa_in)
print('y_gwa_entrance:' , y_gwa_in)
print('z_gwa_entrance:' , z_gwa_in)

print('diff x_gwa_entrance', x_gwa_in - t['xdispIn'])
print('diff y_gwa_entrance', y_gwa_in - t['ydispIn'])

2018-06-20 11:03:59,586 - stpipe - INFO - gwa_ytilt is None deg
2018-06-20 11:03:59,587 - stpipe - INFO - gwa_xtilt is None deg
2018-06-20 11:03:59,587 - stpipe - INFO - gwa_xtilt not applied
2018-06-20 11:03:59,588 - stpipe - INFO - gwa_ytilt not applied


x_gwa_entrance: [ 0.20045291  0.20045681  0.20046072  0.20046463  0.20046853]
y_gwa_entrance: [-0.33840909 -0.33834733 -0.33828557 -0.3382238  -0.33816203]
z_gwa_entrance: [ 0.91940085  0.91942273  0.9194446   0.91946647  0.91948834]
diff x_gwa_entrance      xdispIn      
------------------
 -0.00066847607609
-0.000668491925767
-0.000668507772688
-0.000668523616855
-0.000668539458266
diff y_gwa_entrance      ydispIn      
------------------
-5.12025786775e-06
-5.12037899669e-06
-5.12050010498e-06
-5.12062119212e-06
-5.12074225867e-06


In [11]:
# Slit to GWA out
slit2gwa = slit_wcs.get_transform('slit_frame', 'gwa')
x_gwa_out, y_gwa_out, z_gwa_out = slit2gwa(slitx, slity, lam)
print('x_gwa_exit:' , x_gwa_out)
print('y_gwa_exit:' , y_gwa_out)
print('z_gwa_exit:' , z_gwa_out)

print('diff x_gwa_exit', x_gwa_out - t['xdispLaw'])
print('diff y_gwa_exit', y_gwa_out - t['ydispLaw'])

x_gwa_exit: [ 0.05883232  0.10263868  0.14644503  0.19025139  0.24657497]
y_gwa_exit: [ 0.33840909  0.33834733  0.33828557  0.3382238   0.33816203]
z_gwa_exit: [ 0.93915816  0.93540707  0.92957879  0.92163391  0.908211  ]
diff x_gwa_exit      xdispLaw    
-----------------
 0.00066847607609
0.000668491925767
0.000668507772688
0.000668523616855
0.000668539458266
diff y_gwa_exit      ydispLaw    
-----------------
5.12025786775e-06
5.12037899669e-06
5.12050010498e-06
5.12062119212e-06
5.12074225867e-06


In [12]:
# CAMERA entrance (assuming direction is from sky to detector)
angles = [disperser['theta_x'], disperser['theta_y'],
          disperser['theta_z'], disperser['tilt_y']]
rotation = models.Rotation3DToGWA(angles, axes_order="xyzy", name='rotation')
dircos2unitless = models.DirCos2Unitless()

gwa2cam = rotation.inverse | dircos2unitless
x_camera_entrance, y_camera_entrance = gwa2cam(x_gwa_out, y_gwa_out, z_gwa_out)
print('x_camera_entrance:' , x_camera_entrance)
print('y_camera_entrance:' , y_camera_entrance)

print('diff x_camera_enrtance', x_camera_entrance - t['xcamCosi'])
print('diff y_camera_entrance', y_camera_entrance - t['ycamCosi'])

x_camera_entrance: [-0.08909869 -0.04216234  0.00482219  0.05217109  0.11412658]
y_camera_entrance: [ 0.36136011  0.35978315  0.35899585  0.35900545  0.36025069]
diff x_camera_enrtance     xcamCosi    
----------------
0.00145033329697
0.00144423734841
0.00144609767317
0.00145612214208
 0.0014818101157
diff y_camera_entrance      ycamCosi     
------------------
 -4.6621666802e-05
-2.21757837675e-05
 2.18187125328e-06
 2.68602083252e-05
 5.97757993593e-05


In [13]:
# at FPA
camera = datamodels.CameraModel(refs['camera'])
x_fpa, y_fpa = camera.model.inverse(x_camera_entrance, y_camera_entrance)
print('x_fpa: ', x_fpa )
print('y_fpa: ' , y_fpa)

print('diff x_fpa', x_fpa - t['xfpapos'])
print('diff y_fpa', y_fpa - t['yfpapos'])

x_fpa:  [-0.02553711 -0.01247944  0.00064876  0.01387314  0.03106437]
y_fpa:  [ 0.01741811  0.01725733  0.01719534  0.01722958  0.0174328 ]
diff x_fpa      xfpapos     
-----------------
0.000402029475145
0.000402960277387
0.000404280916169
0.000405890948642
0.000408725158212
diff y_fpa      yfpapos      
------------------
-2.82745717221e-06
 3.43960246917e-07
 3.30588920062e-06
 6.29173449994e-06
  1.0700772956e-05


In [14]:
# at SCA
slit2sca = slit_wcs.get_transform('slit_frame', 'sca')
x_sca_nrs1, y_sca_nrs1 = slit2sca(slitx, slity, lam)

# At NRS2
fpa = datamodels.FPAModel(refs['fpa'])
x_sca_nrs2, y_sca_nrs2 = fpa.nrs2_model.inverse(x_fpa, y_fpa)
print('x_sca1: ', x_sca_nrs1)
print('y_sca1: ' , y_sca_nrs1)
print('x_sca2: ', x_sca_nrs2 )
print('y_sca2: ' , y_sca_nrs2)

print('diff x_sca1', x_sca_nrs1[:2] - t['i'][:2])
print('diff y_sca1', y_sca_nrs1[:2] - t['j'][:2])
print('\n')
print('diff x_sca2', x_sca_nrs2[2:] - t['i'][2:])
print('diff y_sca2', y_sca_nrs2[2:] - t['j'][2:])
print(t['SCA'])

x_sca1:  [  701.8736012   1427.29965691  2156.6444643   2891.33190661  3846.40028601]
y_sca1:  [ 1991.17297878  1982.24032509  1978.79690089  1980.6991596   1991.98890075]
x_sca2:  [ 3539.33136286  2813.90544065  2084.56068477  1349.87321413   394.80466622]
y_sca2:  [ 55.73780147  64.6812901   68.13560776  66.24432233  54.96884604]
diff x_sca1       i      
-------------
21.3349708414
 21.386682077
diff y_sca1        j       
---------------
 -1.15708095401
-0.980891097393


diff x_sca2       i       
--------------
-23.4600536389
 -23.549502365
-23.7069621107
diff y_sca2       j       
--------------
-1.18332504863
-1.34920400705
-1.59414823616
 SCA  
------
SCA491
SCA491
SCA492
SCA492
SCA492


In [15]:
# at oteip
slit2oteip = slit_wcs.get_transform('slit_frame', 'oteip')
x_oteip, y_oteip, _ = slit2oteip(slitx, slity, lam)
print('x_oteip: ', x_oteip)
print('y_oteip: ' , y_oteip)

print('diff x_oteip', x_oteip - t['xOTEIP'])
print('diff y_oteip', y_oteip - t['yOTEIP'])

x_oteip:  [ 0.00356309  0.00361242  0.00366175  0.00371108  0.00376076]
y_oteip:  [-0.0901622  -0.09010688 -0.09005155 -0.08999622 -0.08994092]
diff x_oteip       xOTEIP      
------------------
 -7.8062556419e-18
-2.86229373536e-17
 6.07153216592e-18
 -7.8062556419e-18
 3.38271077815e-17
diff y_oteip       yOTEIP      
------------------
-1.38777878078e-17
               0.0
-1.38777878078e-17
               0.0
               0.0


In [16]:
# at v2, v3 [in arcsec]
slit2v23 = slit_wcs.get_transform('slit_frame', 'v2v3')
v2, v3, _ = slit2v23(slitx, slity, lam)
v2 /= 3600
v3 /= 3600
print('v2: ', v2)
print('v3: ' , v3)

print('diff V2', v2 - t['xV2V3'])
print('diff V3', v3 - t['yV2V3'])

v2:  [ 0.10400871  0.10398709  0.10396548  0.10394386  0.1039221 ]
v3:  [-0.1584595  -0.15843537 -0.15841123 -0.15838709 -0.15836296]
diff V2       xV2V3       
------------------
 1.38777878078e-17
 1.38777878078e-17
               0.0
               0.0
-2.77555756156e-17
diff V3       yV2V3       
------------------
-2.77555756156e-17
               0.0
               0.0
               0.0
               0.0


In [17]:
# Coordinates at Collimator exit
# Applies the Collimator forward transform to MSa absolute coordinates
col = datamodels.open(refs['collimator'])
colx, coly = col.model.inverse(msax, msay)
print('x_collimator_exit', colx)
print('y_collimator_exit', coly)
col.close()

print('diff x_collimator_exit', colx - t['xcoll'])
print('diff y_collimator_exit', coly - t['ycoll'])


x_collimator_exit [ 0.05791667  0.05791633  0.05791599  0.05791564  0.0579153 ]
y_collimator_exit [-0.36119052 -0.36111622 -0.36104192 -0.36096762 -0.36089332]
diff x_collimator_exit       xcoll       
------------------
-6.93889390391e-18
-6.93889390391e-18
-1.38777878078e-17
-6.93889390391e-18
-1.38777878078e-17
diff y_collimator_exit       ycoll       
------------------
               0.0
 5.55111512313e-17
-5.55111512313e-17
 5.55111512313e-17
-1.11022302463e-16


In [18]:
# Save results to an asdf file

fa = AsdfFile()
fa.tree['slitx'] = list(slitx)
fa.tree['slity'] = list(slity)
fa.tree['lam'] = list(lam)
fa.tree['msax'] = list(msax)
fa.tree['msay'] = list(msay)
fa.tree['x_collimator_exit'] = list(colx)
fa.tree['y_collimator_exit'] = list(coly)
fa.tree['x_gwa_entrance'] = list(x_gwa_in)
fa.tree['y_gwa_entrance'] = list(y_gwa_in)
fa.tree['z_gwa_entrance'] = list(z_gwa_in)
fa.tree['x_gwa_exit'] = list(x_gwa_out)
fa.tree['y_gwa_exit'] = list(y_gwa_out)
fa.tree['z_gwa_exit'] = list(z_gwa_out)
fa.tree['x_camera_entrance'] = list(x_camera_entrance)
fa.tree['y_camera_entrance'] = list(y_camera_entrance)
fa.tree['x_fpa'] = list(x_fpa)
fa.tree['y_fpa'] = list(y_fpa)
fa.tree['x_sca_nrs1'] = list(x_sca_nrs1)
fa.tree['y_sca_nrs1'] = list(y_sca_nrs1)
fa.tree['x_sca_nrs2'] = list(x_sca_nrs2)
fa.tree['y_sca_nrs2'] = list(y_sca_nrs2)
fa.tree['x_oteip'] = list(x_oteip)
fa.tree['y_oteip'] = list(y_oteip)
fa.tree['v2'] = list(v2)
fa.tree['v3'] = list(v3)
fa.write_to("msa_functional.asdf", all_array_storage="internal")